In [2]:
%load_ext autoreload
%autoreload 2

import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import sys

sys.path.insert(1, '../preprocessing')
from preprocessing_functions import *
from process_modelling_data import get_modelling_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
start_time = time.time()
accident_df = pd.read_csv('https://www.dropbox.com/s/mdw2asjrh8bm038/US_Accidents_Dec20_updated.csv?dl=1')
download_time = time.time() - start_time
print(f"Data Download Time {download_time} secs")

Data Download Time 41.16887712478638 secs


In [6]:
city_list = ['Phoenix', 'Los Angeles', 'New York', 'Philadelphia', 'Houston', 'Chicago']
state_list = ['AZ', 'CA', 'NY', 'PA', 'TX', 'IL']
six_cities_df = isolate_city_state(accident_df, city_list, state_list)

In [8]:
# Perform OLS Imputation
six_cities_df["Wind_Temp"] = six_cities_df['Wind_Chill(F)'].fillna((six_cities_df['Temperature(F)']*1.0178 - six_cities_df['Wind_Speed(mph)']*0.3023))
six_cities_df = six_cities_df.drop("Wind_Chill(F)", axis=1)
env_vars_numeric = ['Temperature(F)', 'Wind_Temp', 'Humidity(%)', 'Pressure(in)', 
                    'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']

# Perform KNN Imputation
start_time = time.time()
Imputed_env_vars = knn_imputer(subset_df(six_cities_df, env_vars_numeric), k=3)
knn_imputation_time = time.time() - start_time
print(f"KNN Imputation Time {knn_imputation_time / 60} mins")

# Perform Basic Imputation
Imputed_env_vars=basic_impute(Imputed_env_vars)
six_cities_df[env_vars_numeric] = Imputed_env_vars

KNN Imputation Time 2.595370348294576 mins


/Users/sanjeevsingh/Dropbox/Courses/DSE511/Project-3/DSE511-Project-3-Code-Repo/src/preprocessing/preprocessing_functions.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols] = vals_arr
/Users/sanjeevsingh/opt/anaconda3/envs/py3/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/sanjeevsingh/opt/anaconda3/envs/py3/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [9]:
## Impute Categorical Variables
six_cities_df['Wind_Direction'] = six_cities_df['Wind_Direction'].fillna(six_cities_df['Wind_Direction'].mode()[0])
six_cities_df['Weather_Condition'] = six_cities_df['Weather_Condition'].fillna(six_cities_df['Weather_Condition'].mode()[0])
six_cities_df.isna().sum() / six_cities_df.shape[0] * 100

ID                        0.000000
Severity                  0.000000
Start_Time                0.000000
End_Time                  0.000000
Start_Lat                 0.000000
Start_Lng                 0.000000
End_Lat                   0.000000
End_Lng                   0.000000
Distance(mi)              0.000000
Description               0.000000
Number                   75.159532
Street                    0.000000
Side                      0.000000
City                      0.000000
County                    0.000000
State                     0.000000
Zipcode                   0.000000
Country                   0.000000
Timezone                  0.000000
Airport_Code              0.000000
Weather_Timestamp         0.798487
Temperature(F)            0.000000
Humidity(%)               0.000000
Pressure(in)              0.000000
Visibility(mi)            0.000000
Wind_Direction            0.000000
Wind_Speed(mph)           0.000000
Precipitation(in)         0.000000
Weather_Condition   

In [50]:
six_cities_df_X = six_cities_df.copy()
X_train, X_test, y_train, y_test = encode_std_extract_split(six_cities_df_X)

/Users/sanjeevsingh/Dropbox/Courses/DSE511/Project-3/DSE511-Project-3-Code-Repo/src/preprocessing/preprocessing_functions.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  print("feature :", numerical_features)
/Users/sanjeevsingh/Dropbox/Courses/DSE511/Project-3/DSE511-Project-3-Code-Repo/src/preprocessing/preprocessing_functions.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for feature in numerical_features:


In [72]:
pca = PCA(n_components=0.95, svd_solver = 'full')
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [68]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

clf = AdaBoostClassifier(n_estimators=800, random_state=0)
clf.fit(X_train_pca, y_train)
y_pred = clf.predict(X_test_pca)
accuracy_score(y_test, y_pred)

0.8210770761915294

In [71]:
X_train_pca.shape

(81378, 29)

In [6]:
(X_train, X_val, X_test, X_train_pca, X_val_pca, X_test_pca, y_train, y_val, y_test) = get_modelling_data()

In [10]:
y_train.value_counts()[0] / y_train.value_counts()[1]

3.3544425716349626

In [11]:
16612 / (55724 + 16612)

0.2296505197965052